### Q1. Getting the embeddings model

In [30]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [31]:
user_question = "I just discovered the course. Can I still join it?"
vector_user_question = embedding_model.encode(user_question)
vector_user_question[0]

0.07822263

In [32]:
dims = len(vector_user_question)

In [33]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [34]:
course = "machine-learning-zoomcamp"
course_documents = [doc for doc in documents if doc["course"] == course]
len(course_documents)

375

### Q2. Creating the embeddings

In [35]:
embeddings = []
for doc in course_documents:
    qa_text = f'{doc['question']} {doc['text']}'
    qa_embedding = embedding_model.encode(qa_text)
    embeddings.append(qa_embedding)

In [36]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

### Q3. Search

In [37]:
v = vector_user_question
scores = X.dot(v)
np.max(scores)

0.6506573

In [38]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=course_documents, embeddings=X)
vec_search_results = search_engine.search(v, num_results=5)

### Q4. Hit-rate for our search engine

In [39]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [40]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [41]:
from tqdm.auto import tqdm

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    vector_question = embedding_model.encode(q['question'])
    results = search_engine.search(vector_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

hit_rate(relevance_total)

  0%|          | 0/1830 [00:00<?, ?it/s]

0.9398907103825137

### Q5. Indexing with Elasticsearch

In [42]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '29c3a5c13299', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5cKUWYxkRO24GR54w_C6ag', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [43]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": dims, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "machine-learning-zoomcamp-course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'machine-learning-zoomcamp-course-questions'})

In [44]:
docs_with_vectors = []
for i, doc in enumerate(course_documents):
    doc["text_vector"] = embeddings[i].tolist()
    docs_with_vectors.append(doc)

for doc in docs_with_vectors:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [45]:
query = {
    "field": "text_vector",
    "query_vector": vector_user_question,
    "k": 5,
    "num_candidates": 10000, 
}

res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"])
res["hits"]["hits"][0]['_source']['id']

'ee58a693'

### Q6. Hit-rate for Elasticsearch

In [46]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    vector_question = embedding_model.encode(q['question'])
    query = {
        "field": "text_vector",
        "query_vector": vector_question,
        "k": 5,
        "num_candidates": 10000, 
    }
    results = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"])
    results = results["hits"]["hits"]
    relevance = [d['_source']['id'] == doc_id for d in results]
    relevance_total.append(relevance)

hit_rate(relevance_total)

  0%|          | 0/1830 [00:00<?, ?it/s]

0.9398907103825137